In [6]:
from dotenv import load_dotenv
import os
import boto3
import botocore

load_dotenv() 

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.getenv('AWS_S3_ENDPOINT_URL')
region_name = os.getenv('AWS_DEFAULT_REGION')
bucket_name = os.getenv('AWS_S3_BUCKET')


if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or more connection variables are empty.  "
                     "Please check your connection to an S3 bucket.")

print(f"Connecting to S3 at {endpoint_url} using {aws_access_key_id} in bucket: {bucket_name}")

session = boto3.session.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

s3_resource = session.resource(
        's3',
        config=botocore.client.Config(signature_version='s3v4'), 
        endpoint_url=endpoint_url,
        region_name=region_name
    )

bucket = s3_resource.Bucket(bucket_name)

def upload_directory_to_s3(local_directory, s3_prefix):
    print (f"{local_directory}, {s3_prefix}")
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            print(filename)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)


# upload_directory_to_s3("/home/dev/.cache/huggingface/hub/models--ibm-granite--granite-3.1-8b-instruct/snapshots/4009206d5fc95d2e65a7b7633e159d6e97e25d35", "models/ibm-granite/mmodels--ibm-granite--granite-3.1-8b-instruct")

upload_directory_to_s3("/workspaces/ai-lab/training/models/upload", "ggufs")

Connecting to S3 at https://minio-s3-s3storage.apps.cluster-gqbjb.gqbjb.sandbox134.opentlc.com using NNMZdJkgBOPIPaZj in bucket: mymodels
/workspaces/ai-lab/training/models/upload, ggufs
granite-3.3-2b-instruct-Q4_K_M.gguf
/workspaces/ai-lab/training/models/upload/granite-3.3-2b-instruct-Q4_K_M.gguf -> ggufs/granite-3.3-2b-instruct-Q4_K_M.gguf
trained_granite_7B-Q4_K_M.gguf
/workspaces/ai-lab/training/models/upload/trained_granite_7B-Q4_K_M.gguf -> ggufs/trained_granite_7B-Q4_K_M.gguf
